In [3]:
import numpy as np
import xarray as xr
import dask
import warnings

### Preliminaries

In [1]:
###############################
# Set paths
# UPDATE THIS FOR REPRODUCTION
###############################
nex_in = '/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/nex-gddp/'
cil_in = '/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/cil-gdpcir/'
isi_in = '/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/isimip3b/regridded/conservative/'
cbp_in = '/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/metrics/carbonplan/regridded/conservative/'

out_path = '/gpfs/group/kaf26/default/dcl5300/lafferty-sriver_inprep_tbh_DATA/uc_results/'

In [2]:
###################
# Models
###################
from utils import nex_ssp_dict, cil_ssp_dict, isimip_ssp_dict, gardsv_ssp_dict, deepsdbc_dict

nex_models = list(nex_ssp_dict.keys())
cil_models = list(cil_ssp_dict.keys())
isi_models = list(isimip_ssp_dict.keys())
cbp_gard_models = list(gardsv_ssp_dict.keys())
cbp_deep_models = list(deepsdbc_dict.keys())

In [4]:
ds = xr.open_dataset(nex_in + 'avg' + '/' + nex_models[0] + '.nc')

In [4]:
############
# Dask
############
from dask_jobqueue import PBSCluster

cluster = PBSCluster(cores=1, memory='40GB', resource_spec='pmem=40GB',
                     account='open',
                     worker_extra_args=['#PBS -l feature=rhel7'], 
                     walltime='00:10:00')

cluster.scale(jobs=30)  # ask for jobs

from dask.distributed import Client
client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.102.201.236:32914,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Total uncertainty

In [5]:
#######################################################################
# Total uncertainty: variance across all models, scenarios, ensembles 
#######################################################################
def uc_total(nex_in, nex_models, 
             cil_in, cil_models, 
             isi_in, isi_models, 
             cbp_in, cbp_gard_models, cbp_deep_models,
             metric, year):
    ######################
    # Read all ensembles
    ######################
    # NEX-GDDP 
    ds_out = []
    for model in nex_models:
        ds = xr.open_dataset(nex_in + metric + '/' + model + '.nc')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=year)
        ds['lon'] = np.where(ds['lon'] > 180, ds['lon'] - 360, ds['lon'])
        ds = ds.sortby('lon')
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(ensemble = 'NEX')
        ds = ds.assign_coords(model = ds.encoding['source'].replace(nex_in, '').split('/')[-1][:-3])
        ds_out.append(ds)
    ds_nex = xr.concat(ds_out, dim='model', compat='identical')

    # CIL-GDPCIR
    ds_out = []
    for model in cil_models:
        ds = xr.open_dataset(cil_in + metric + '/' + model, engine='zarr')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=year)
        ds = ds.sel(lat=slice(-60, 90))
        ds = ds.assign_coords(ensemble = 'CIL')
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(model = ds.encoding['source'].replace(cil_in, '').split('/')[-1])
        ds_out.append(ds)
    ds_cil = xr.concat(ds_out, dim='model', compat='identical')

    # ISIMIP
    ds_out = []
    for model in isi_models:
        ds = xr.open_dataset(isi_in + metric + '/' + model + '.nc')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=year)
        ds['lon'] = np.where(ds['lon'] > 180, ds['lon'] - 360, ds['lon'])
        ds = ds.sortby('lon')
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(ensemble = 'ISIMIP')
        ds = ds.assign_coords(model = ds.encoding['source'].replace(isi_in, '').split('/')[-1][:-3])
        ds_out.append(ds)
    ds_isi = xr.concat(ds_out, dim='model', compat='identical')

    # carbonplan: GARD-SV
    ds_out = []
    for model in cbp_gard_models:
        ds = xr.open_dataset(cbp_in + 'GARD-SV/' + metric + '/' + model + '.nc')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=year)
        ds = ds.sel(lat=slice(-60, 90))
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(ensemble = 'GARD-SV')
        ds = ds.assign_coords(model = ds.encoding['source'].replace(cbp_in, '').split('/')[-1][:-3])
        # for some models/methods we are missing 
        # precip so need to fill with NaNs
        if 'pr' not in ds.data_vars:
            ds['pr'] = xr.full_like(ds['tas'], np.NaN)
        ds_out.append(ds)
    ds_cbp_gard = xr.concat(ds_out, dim='model', compat='identical')
    
    # carbonplan: DeepSD-BC
    ds_out = []
    for model in cbp_deep_models:
        ds = xr.open_dataset(cbp_in + 'DeepSD-BC/' + metric + '/' + model + '.nc')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=year)
        ds = ds.sel(lat=slice(-60, 90))
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(ensemble = 'DeepSD-BC')
        ds = ds.assign_coords(model = ds.encoding['source'].replace(cbp_in, '').split('/')[-1][:-3])
        # for some models/methods we are missing 
        # precip so need to fill with NaNs
        if 'pr' not in ds.data_vars:
            ds['pr'] = xr.full_like(ds['tas'], np.NaN)
        ds_out.append(ds)
    ds_cbp_deep = xr.concat(ds_out, dim='model', compat='identical')

    ###########################
    # Merge all and mask ocean
    ###########################
    ds = xr.concat([ds_nex, ds_cil, ds_isi, ds_cbp_gard, ds_cbp_deep],
                       dim='ensemble', fill_value=np.nan)
    
    # mask out ocean points (NEX is only available over land)
    ds_mask = ds.sel(ensemble='NEX').isel(ssp=0, model=0)[list(ds.keys())[0]].isnull()
    ds = xr.where(ds_mask, np.nan, ds)
    
    ##########################
    # Uncertainty calculation
    ##########################
    ## Total uncertainty
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        U_total_true = ds.var(dim=['ensemble', 'ssp', 'model']) # throws warning when all NaNs

    U_total_true = U_total_true.assign_coords(uncertainty = 'total_true')
    
    return U_total_true

## Annual averages

In [6]:
metric = 'annual_avgs'

In [ ]:
%%time
################################
# UC on raw outputs (no iav)
################################
delayed_res = []
for year in range(2015, 2100):
    # Read all ensembles and compute UC
    tmp_res = dask.delayed(uc_total)(nex_in, nex_models, 
                                     cil_in, cil_models, 
                                     isi_in, isi_models, 
                                     cbp_in, cbp_gard_models, cbp_deep_models,
                                     metric, year)
    
    # Append
    delayed_res.append(tmp_res)
    
# Compute
res = dask.compute(*delayed_res)

# Merge and store
ds_out = xr.concat(res, dim='time')
ds_out.to_netcdf(out_path + 'total_uncertainty/' + metric +'.nc')

## Annual maxs

In [9]:
metric = 'annual_maxs'

In [10]:
%%time
################################
# UC on raw outputs (no iav)
################################
delayed_res = []
for year in range(2015, 2100):
    # Read all ensembles and compute UC
    tmp_res = dask.delayed(uc_total)(nex_in, nex_models, 
                                     cil_in, cil_models, 
                                     isi_in, isi_models, 
                                     cbp_in, cbp_gard_models, cbp_deep_models,
                                     metric, year)
    
    # Append
    delayed_res.append(tmp_res)
    
# Compute
res = dask.compute(*delayed_res)

# Merge and store
ds_out = xr.concat(res, dim='time')
ds_out.to_netcdf(out_path + 'total_uncertainty/' + metric +'.nc')

CPU times: user 1min 8s, sys: 5.73 s, total: 1min 13s
Wall time: 8min 10s


# No interannual variability

In [5]:
################################################################
# Uncertainty characterization following Hawkins & Sutton 2009 
# No consideration of internal variability!
################################################################
def uc_hs09(nex_in, nex_models, 
            cil_in, cil_models, 
            isi_in, isi_models, 
            cbp_in, cbp_gard_models, cbp_deep_models,
            metric, year):
    ##################################
    # Read and format all ensembles
    ##################################
    # NEX-GDDP 
    ds_out = []
    for model in nex_models:
        ds = xr.open_dataset(nex_in + metric + '/' + model + '.nc')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=year)
        ds['lon'] = np.where(ds['lon'] > 180, ds['lon'] - 360, ds['lon'])
        ds = ds.sortby('lon')
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(ensemble = 'NEX')
        ds = ds.assign_coords(model = ds.encoding['source'].replace(nex_in, '').split('/')[-1][:-3])
        ds_out.append(ds)
    ds_nex = xr.concat(ds_out, dim='model', compat='identical')

    # CIL-GDPCIR
    ds_out = []
    for model in cil_models:
        ds = xr.open_dataset(cil_in + metric + '/' + model, engine='zarr')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=year)
        ds = ds.sel(lat=slice(-60, 90))
        ds = ds.assign_coords(ensemble = 'CIL')
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(model = ds.encoding['source'].replace(cil_in, '').split('/')[-1])
        ds_out.append(ds)
    ds_cil = xr.concat(ds_out, dim='model', compat='identical')

    # ISIMIP
    ds_out = []
    for model in isi_models:
        ds = xr.open_dataset(isi_in + metric + '/' + model + '.nc')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=year)
        ds['lon'] = np.where(ds['lon'] > 180, ds['lon'] - 360, ds['lon'])
        ds = ds.sortby('lon')
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(ensemble = 'ISIMIP')
        ds = ds.assign_coords(model = ds.encoding['source'].replace(isi_in, '').split('/')[-1][:-3])
        ds_out.append(ds)
    ds_isi = xr.concat(ds_out, dim='model', compat='identical')

    # carbonplan: GARD-SV
    ds_out = []
    for model in cbp_gard_models:
        ds = xr.open_dataset(cbp_in + 'GARD-SV/' + metric + '/' + model + '.nc')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=year)
        ds = ds.sel(lat=slice(-60, 90))
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(ensemble = 'GARD-SV')
        ds = ds.assign_coords(model = ds.encoding['source'].replace(cbp_in, '').split('/')[-1][:-3])
        # for some models/methods we are missing 
        # precip so need to fill with NaNs
        if 'pr' not in ds.data_vars:
            ds['pr'] = xr.full_like(ds['tas'], np.NaN)
        ds_out.append(ds)
    ds_cbp_gard = xr.concat(ds_out, dim='model', compat='identical')
    
    # carbonplan: DeepSD-BC
    ds_out = []
    for model in cbp_deep_models:
        ds = xr.open_dataset(cbp_in + 'DeepSD-BC/' + metric + '/' + model + '.nc')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=year)
        ds = ds.sel(lat=slice(-60, 90))
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(ensemble = 'DeepSD-BC')
        ds = ds.assign_coords(model = ds.encoding['source'].replace(cbp_in, '').split('/')[-1][:-3])
        # for some models/methods we are missing 
        # precip so need to fill with NaNs
        if 'pr' not in ds.data_vars:
            ds['pr'] = xr.full_like(ds['tas'], np.NaN)
        ds_out.append(ds)
    ds_cbp_deep = xr.concat(ds_out, dim='model', compat='identical')

    ###########################
    # Merge all and mask ocean
    ###########################
    ds = xr.concat([ds_nex, ds_cil, ds_isi, ds_cbp_gard, ds_cbp_deep],
                       dim='ensemble', fill_value=np.nan)
    
    # mask out ocean points (NEX is only available over land)
    ds_mask = ds.sel(ensemble='NEX').isel(ssp=0, model=0)[list(ds.keys())[0]].isnull()
    ds = xr.where(ds_mask, np.nan, ds)
    
    ##########################
    # Uncertainty calculation
    ##########################
    ##  Model uncertainty
    # Variance across models, averaged over scenarios and ensembles
    U_model = ds.var(dim='model')
    weights = ds.isel(lat=300, lon=800)[list(ds.data_vars)[0]].count(dim='model').rename('weights')     # weights (choose point over land)
    weights = xr.where(weights == 1, 0, weights) # remove combinations where variance was calculated over 1 entry
    U_model = U_model.weighted(weights).mean(dim=['ssp', 'ensemble']) # weighted average

    ## Scenario uncertainty
    # Variance across multi-model means (HS09 approach)
    U_scen = ds.mean(dim=['model', 'ensemble']).var(dim='ssp')

    ## Downscaling uncertainy
    # Variance across ensembles, averaged over models and scenarios
    U_ens = ds.var(dim='ensemble')
    weights = ds.isel(lat=300, lon=800)[list(ds.data_vars)[0]].count(dim='ensemble').rename('weights') # weights
    weights = xr.where(weights == 1, 0, weights) # remove combinations where variance was calculated over 1 entry
    U_ens = U_ens.weighted(weights).mean(dim=['ssp', 'model'])

    ## Total uncertainty    
    # Our 'simulated' total uncertainty
    # This will in general not equal true total
    U_total = U_model + U_scen + U_ens

    ## Merge and return
    U_model = U_model.assign_coords(uncertainty = 'model')
    U_scen = U_scen.assign_coords(uncertainty = 'scenario')
    U_ens = U_ens.assign_coords(uncertainty = 'ensemble')
    U_total = U_total_sim.assign_coords(uncertainty = 'total')
    
    return xr.concat([U_model, U_scen, U_ens, U_total, U_total], dim='uncertainty')

## Annual averages

In [12]:
metric = 'annual_avgs'

In [13]:
%%time
################################
# UC on raw outputs (no iav)
################################
delayed_res = []
for year in range(2015, 2101):
    # Read all ensembles and compute UC
    tmp_res = dask.delayed(uc_hs09)(nex_in, nex_models, 
                                    cil_in, cil_models, 
                                    isi_in, isi_models, 
                                    cbp_in, cbp_gard_models, cbp_deep_models,
                                    metric, year)
    
    # Append
    delayed_res.append(tmp_res)
    
# Compute
res = dask.compute(*delayed_res)

# Merge and store
ds_out = xr.concat(res, dim='time')
ds_out.to_netcdf(out_path + 'hs09_no_iav/' + metric +'.nc')

## Annual maxs

In [8]:
metric = 'annual_maxs'

In [8]:
%%time
################################
# UC on raw outputs (no iav)
################################
delayed_res = []
for year in range(2015, 2101):
    # Read all ensembles and compute UC
    tmp_res = dask.delayed(uc_hs09)(nex_in, nex_models, 
                                    cil_in, cil_models, 
                                    isi_in, isi_models, 
                                    cbp_in, cbp_gard_models, cbp_deep_models,
                                    metric, year)
    
    # Append
    delayed_res.append(tmp_res)
    
# Compute
res = dask.compute(*delayed_res)

# Merge and store
ds_out = xr.concat(res, dim='time')
ds_out.to_netcdf(out_path + 'hs09_no_iav/' + metric +'.nc')

CPU times: user 15.4 s, sys: 9.44 s, total: 24.8 s
Wall time: 7min 10s


# Interannual variability

In [5]:
################################################################
# Uncertainty characterization following Hawkins & Sutton 2009 
# 'Forced response' = 10 year rolling mean
################################################################
def uc_hs09_forced(nex_in, nex_models, 
                   cil_in, cil_models, 
                   isi_in, isi_models, 
                   cbp_in, cbp_gard_models, cbp_deep_models,
                   metric, year):
    ######################
    # Read all ensembles
    ######################
    # NEX-GDDP 
    ds_out = []
    for model in nex_models:
        ds = xr.open_dataset(nex_in + metric + '/' + model + '.nc')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=slice(year-6, year+6)) # faster rolling mean
        ds = ds.assign_coords(model = ds.encoding['source'].replace(nex_in, '').split('/')[-1][:-3])        
        ds = ds.rolling(time=10, center=True).mean().sel(time=year)
        ds['lon'] = np.where(ds['lon'] > 180, ds['lon'] - 360, ds['lon'])
        ds = ds.sortby('lon')
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(ensemble = 'NEX')
        ds_out.append(ds)
    ds_nex = xr.concat(ds_out, dim='model', compat='identical')

    # CIL-GDPCIR
    ds_out = []
    for model in cil_models:
        ds = xr.open_dataset(cil_in + metric + '/' + model, engine='zarr')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=slice(year-6, year+6)) # faster rolling mean
        ds = ds.assign_coords(model = ds.encoding['source'].replace(cil_in, '').split('/')[-1])        
        ds = ds.rolling(time=10, center=True).mean().sel(time=year)
        ds = ds.sel(lat=slice(-60, 90))
        ds = ds.assign_coords(ensemble = 'CIL')
        ds = ds.sortby('ssp')
        ds_out.append(ds)
    ds_cil = xr.concat(ds_out, dim='model', compat='identical')

    # ISIMIP
    ds_out = []
    for model in isi_models:
        ds = xr.open_dataset(isi_in + metric + '/' + model + '.nc')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=slice(year-6, year+6)) # faster rolling mean
        ds = ds.assign_coords(model = ds.encoding['source'].replace(isi_in, '').split('/')[-1][:-3])        
        ds = ds.rolling(time=10, center=True).mean().sel(time=year)
        ds['lon'] = np.where(ds['lon'] > 180, ds['lon'] - 360, ds['lon'])
        ds = ds.sortby('lon')
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(ensemble = 'ISIMIP')
        ds_out.append(ds)
    ds_isi = xr.concat(ds_out, dim='model', compat='identical')

    # carbonplan: GARD-SV
    ds_out = []
    for model in cbp_gard_models:
        ds = xr.open_dataset(cbp_in + 'GARD-SV/' + metric + '/' + model + '.nc')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=slice(year-6, year+6)) # faster rolling mean        
        ds = ds.assign_coords(model = ds.encoding['source'].replace(cbp_in, '').split('/')[-1][:-3])        
        ds = ds.rolling(time=10, center=True).mean().sel(time=year)
        ds = ds.sel(lat=slice(-60, 90))
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(ensemble = 'GARD-SV')
        # for some models/methods we are missing 
        # precip so need to fill with NaNs
        if 'pr' not in ds.data_vars:
            ds['pr'] = xr.full_like(ds['tas'], np.NaN)
        ds_out.append(ds)
    ds_cbp_gard = xr.concat(ds_out, dim='model', compat='identical')
    
    # carbonplan: DeepSD-BC
    ds_out = []
    for model in cbp_deep_models:
        ds = xr.open_dataset(cbp_in + 'DeepSD-BC/' + metric + '/' + model + '.nc')
        ds['time'] = ds.indexes['time'].year
        ds = ds.sel(time=slice(year-6, year+6)) # faster rolling mean        
        ds = ds.assign_coords(model = ds.encoding['source'].replace(cbp_in, '').split('/')[-1][:-3])        
        ds = ds.rolling(time=10, center=True).mean().sel(time=year)
        ds = ds.sel(lat=slice(-60, 90))
        ds = ds.sortby('ssp')
        ds = ds.assign_coords(ensemble = 'DeepSD-BC')
        # for some models/methods we are missing 
        # precip so need to fill with NaNs
        if 'pr' not in ds.data_vars:
            ds['pr'] = xr.full_like(ds['tas'], np.NaN)
        ds_out.append(ds)
    ds_cbp_deep = xr.concat(ds_out, dim='model', compat='identical')

    ###########################
    # Merge all and mask ocean
    ###########################
    ds = xr.concat([ds_nex, ds_cil, ds_isi, ds_cbp_gard, ds_cbp_deep],
                       dim='ensemble', fill_value=np.nan)
    
    # mask out ocean points (NEX is only available over land)
    ds_mask = ds.sel(ensemble='NEX').isel(ssp=0, model=0)[list(ds.keys())[0]].isnull()
    ds = xr.where(ds_mask, np.nan, ds)
    
    ##########################
    # Uncertainty calculation
    ##########################
    ##  Model uncertainty
    # Variance across models, averaged over scenarios and ensembles
    U_model = ds.var(dim='model')
    weights = ds.isel(lat=300, lon=800)[list(ds.data_vars)[0]].count(dim='model').rename('weights') # weights (choose point over land)
    weights = xr.where(weights == 1, 0, weights) # remove combinations where variance was calculated over 1 entry
    U_model = U_model.weighted(weights).mean(dim=['ssp', 'ensemble']) # weighted average

    ## Scenario uncertainty
    # Variance across multi-model means (HS09 approach)
    U_scen = ds.mean(dim=['model', 'ensemble']).var(dim='ssp')

    ## Downscaling uncertainy
    # Variance across ensembles, averaged over models and scenarios
    U_ens = ds.var(dim='ensemble')
    weights = ds.isel(lat=300, lon=800)[list(ds.data_vars)[0]].count(dim='ensemble').rename('weights') # weights
    weights = xr.where(weights == 1, 0, weights) # remove combinations where variance was calculated over 1 entry
    U_ens = U_ens.weighted(weights).mean(dim=['ssp', 'model'])

    ## Merge and return
    U_model = U_model.assign_coords(uncertainty = 'model')
    U_scen = U_scen.assign_coords(uncertainty = 'scenario')
    U_ens = U_ens.assign_coords(uncertainty = 'ensemble')
    
    return xr.concat([U_model, U_scen, U_ens], dim='uncertainty')

In [6]:
################################################################
# Uncertainty characterization following Hawkins & Sutton 2009 
# Interannual variability (single value for all years)
################################################################

# This function calculates the internal variability (variance
# over all years of residuals from rolling mean) for a given 
# model-ssp-ensemble

def uc_hs09_iav(path_in, ensemble, model, metric):
    
    # read CIL (needs zarr)
    if ensemble == 'CIL':
        ds = xr.open_dataset(path_in + metric + '/' + model, engine='zarr')
    
    # read carbon (different path ordering)
    elif ensemble == 'DeepSD-BC' or ensemble == 'GARD-SV':
        ds = xr.open_dataset(path_in + ensemble + '/' + metric + '/' + model + '.nc')
    
    # read NEX and ISIMIP
    else:
        ds = xr.open_dataset(path_in + metric + '/' + model + '.nc')

    ## Format same for merge
    
    # Year index only
    ds['time'] = ds.indexes['time'].year

    # Add model and ensemble dims
    ds = ds.assign_coords(ensmod = ensemble + '__' + model)
    ds = ds.sortby('ssp')
    
    # Transform longitudes if necessary
    if ds['lon'].max() > 180:
        ds['lon'] = np.where(ds['lon'] > 180, ds['lon'] - 360, ds['lon'])
    ds = ds.sortby('lon')
    
    # Remove Antarctica
    if ds['lat'].min() < -60:
        ds = ds.sel(lat=slice(-60, 90))
    
    # Missing precip for one carbonplan model
    if 'pr' not in ds.data_vars:
            ds['pr'] = xr.full_like(ds['tas'], np.NaN)
            
    ## Get IAV estimate: variance of rolling mean residuals
    ds_rolling = ds.rolling(time=10, center=True).mean().sel(time=slice(2020,2096))
    return (ds - ds_rolling).var(dim='time')



# Make a delayed list with IAV of all models-ssps-ensembles which 
# can then be combined into one dataset and averaged for best estimate

def make_delayed_list_iav(metric):
    # Parallelize with dask over models
    delayed_res = []
    
    # NEX
    for model in nex_models:
        tmp_res = dask.delayed(uc_hs09_iav)(nex_in, 'NEX', model, metric)
        delayed_res.append(tmp_res)
        
    # CIL
    for model in cil_models:
        tmp_res = dask.delayed(uc_hs09_iav)(cil_in, 'CIL', model, metric)
        delayed_res.append(tmp_res)
        
    # ISIMIP
    for model in isi_models:
        tmp_res = dask.delayed(uc_hs09_iav)(isi_in, 'ISIMIP', model, metric)
        delayed_res.append(tmp_res)
        
    # carbonplan GARD-SV
    for model in cbp_gard_models:
        tmp_res = dask.delayed(uc_hs09_iav)(cbp_in, 'GARD-SV', model, metric)
        delayed_res.append(tmp_res)
        
    # carbonplan DeepSD-BC
    for model in cbp_deep_models:
        tmp_res = dask.delayed(uc_hs09_iav)(cbp_in, 'DeepSD-BC', model, metric)
        delayed_res.append(tmp_res)
        
    # return
    return delayed_res

## Annual averages

In [7]:
metric = 'annual_avgs'

In [8]:
%%time
################################
# Interannual variability
################################
delayed_res = make_delayed_list_iav(metric)
    
# Compute
res = dask.compute(*delayed_res)

# Merge and average over ensemble + model (ensmod) and ssp
ds_out = xr.concat(res, dim='ensmod').mean(dim=['ensmod', 'ssp'])
ds_out.to_netcdf(out_path + 'hs09_iav/' + metric +'_iav.nc')

CPU times: user 15.4 s, sys: 9.44 s, total: 24.8 s
Wall time: 7min 10s


In [8]:
%%time
################################
# UC on forced response
################################
delayed_res = []
for year in range(2020, 2097):
    # Read all ensembles and compute UC
    tmp_res = dask.delayed(uc_hs09_forced)(nex_in, nex_models, 
                                          cil_in, cil_models, 
                                          isi_in, isi_models, 
                                          cbp_in, cbp_gard_models, cbp_deep_models,
                                          metric, year)
    
    # Append
    delayed_res.append(tmp_res)
    
# Compute
res = dask.compute(*delayed_res)

# Merge and store
ds_out = xr.concat(res, dim='time')
ds_out.to_netcdf(out_path + 'hs09_iav/' + metric +'.nc')

CPU times: user 3min 24s, sys: 27.5 s, total: 3min 51s
Wall time: 27min 22s


## Annual maxs

In [7]:
metric = 'annual_maxs'

In [8]:
%%time
################################
# Interannual variability
################################
delayed_res = make_delayed_list_iav(metric)
    
# Compute
res = dask.compute(*delayed_res)

# Merge and average over ensemble + model (ensmod) and ssp
ds_out = xr.concat(res, dim='ensmod').mean(dim=['ensmod', 'ssp'])
ds_out.to_netcdf(out_path + 'hs09_iav/' + metric +'_iav.nc')

CPU times: user 49.1 s, sys: 15 s, total: 1min 4s
Wall time: 4min 50s


In [9]:
%%time
################################
# UC on forced response
################################
delayed_res = []
for year in range(2020, 2097):
    # Read all ensembles and compute UC
    tmp_res = dask.delayed(uc_hs09_forced)(nex_in, nex_models, 
                                          cil_in, cil_models, 
                                          isi_in, isi_models, 
                                          cbp_in, cbp_gard_models, cbp_deep_models,
                                          metric, year)
    
    # Append
    delayed_res.append(tmp_res)
    
# Compute
res = dask.compute(*delayed_res)

# Merge and store
ds_out = xr.concat(res, dim='time')
ds_out.to_netcdf(out_path + 'hs09_iav/' + metric +'.nc')

CPU times: user 2min 43s, sys: 26.2 s, total: 3min 9s
Wall time: 17min 22s
